# Recommender System based on Review Book in E-commerce Using word2vec Algorithm


1. Yolanda R.M Manurung (12S17017)
2. Stella Sitinjak      (12S17027)
3. Ekis Naomi Lasma     (12S17059)


### Dataset
Data yang digunakan pada proyek ini berasal dari kaggle (https://www.kaggle.com/gapple/goodreads-book-reviews-20k)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import re
import gensim
from gensim.models import Word2Vec
import numpy as np
from scipy import linalg

In [3]:
data = pd.read_csv("./all_data.csv")
data.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\r\nThat’s the number of times that the wo...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\r\n""Rhys stepped off the stairs and...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


# Data Preprocessing

In [4]:
data.isnull().sum()

book_title                0
Book_series           12779
book_series_url       12779
book_image                2
book_image_url            2
book_rating               0
book_author               0
author_url                0
genre                   121
reviewer_name            25
reviewer_url             25
reviewer_image           25
reviewer_image_url       25
review                  742
ID                        0
dtype: int64

In [5]:
data.dtypes

book_title             object
Book_series            object
book_series_url        object
book_image             object
book_image_url         object
book_rating           float64
book_author            object
author_url             object
genre                  object
reviewer_name          object
reviewer_url           object
reviewer_image         object
reviewer_image_url     object
review                 object
ID                      int64
dtype: object

In [6]:
data['Book_series'] = data['Book_series'].fillna(data['Book_series'].mode().iloc[0])
data['book_series_url'] = data['book_series_url'].fillna(data['book_series_url'].mode().iloc[0])
data['book_image'] = data['book_image'].fillna(data['book_image'].mode().iloc[0])
data['book_image_url'] = data['book_image_url'].fillna(data['book_image_url'].mode().iloc[0])
data['genre'] = data['genre'].fillna(data['genre'].mode().iloc[0])
data['reviewer_name'] = data['reviewer_name'].fillna(data['reviewer_name'].mode().iloc[0])
data['reviewer_url'] = data['reviewer_url'].fillna(data['reviewer_url'].mode().iloc[0])
data['reviewer_image'] = data['reviewer_image'].fillna(data['reviewer_image'].mode().iloc[0])
data['reviewer_image_url'] = data['reviewer_image_url'].fillna(data['reviewer_image_url'].mode().iloc[0])
data['review'] = data['review'].fillna(data['review'].mode().iloc[0])

In [7]:
data.isnull().sum()

book_title            0
Book_series           0
book_series_url       0
book_image            0
book_image_url        0
book_rating           0
book_author           0
author_url            0
genre                 0
reviewer_name         0
reviewer_url          0
reviewer_image        0
reviewer_image_url    0
review                0
ID                    0
dtype: int64

In [8]:
data.duplicated().sum()

0

In [9]:
data['book_title'].duplicated().sum()

20837

In [10]:
data['book_title'].drop_duplicates(keep=False,inplace=True)
data.head()

,book_title,Book_series,book_series_url,book_image,book_image_url,book_rating,book_author,author_url,genre,reviewer_name,reviewer_url,reviewer_image,reviewer_image_url,review,ID
0,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",emma,https://www.goodreads.com/user/show/32879029-emma,https://images.gr-assets.com/users/1520568130p...,https://www.goodreads.com/user/show/32879029-emma,202.\r\nThat’s the number of times that the wo...,1
1,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Em (RunawayWithDreamthieves),https://www.goodreads.com/user/show/42130592-e...,https://images.gr-assets.com/users/1538507799p...,https://www.goodreads.com/user/show/42130592-e...,I need a black silk robe and a balcony to wear...,2
2,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Cait • A Page with a View,https://www.goodreads.com/user/show/47412532-c...,https://images.gr-assets.com/users/1523134490p...,https://www.goodreads.com/user/show/47412532-c...,THIS BOOK TOTALLY WRECKED ME. In the best poss...,3
3,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Natalie Monroe,https://www.goodreads.com/user/show/6688207-na...,https://images.gr-assets.com/users/1339730817p...,https://www.goodreads.com/user/show/6688207-na...,"3.75 stars\r\n""Rhys stepped off the stairs and...",4
4,A Court of Wings and Ruin,(A Court of Thorns and Roses #3),https://www.goodreads.com/series/104014-a-cour...,https://images.gr-assets.com/books/1485528243l...,https://www.goodreads.com/book/photo/23766634-...,4.51,Sarah J. Maas,https://www.goodreads.com/author/show/3433047....,"Fantasy, Romance, Young Adult, New Adult",Emily May,https://www.goodreads.com/user/show/4622890-em...,https://images.gr-assets.com/users/1528390751p...,https://www.goodreads.com/user/show/4622890-em...,I wish so much that I could tell you I loved t...,5


In [11]:
coba = data["review"][0]
coba

"202.\r\nThat’s the number of times that the word “mate,” in any of its hellish forms, is used in this book.\r\nTWO.\r\nHUNDRED.\r\nAND.\r\nTWO.\r\nhttps://emmareadstoomuch.wordpress.co...\r\nThat statistic alone should be enough to justify this one star rating. If you feel satisfied enough to just remove yourself from this review, run with that feeling. Because I have, like. Seven pages of sh*t to complain about.\r\nThat’s if I’m efficient.\r\nAnd I’ve never been an efficient reviewer. Once. In my life. I counted two hundred (and two) uses of a word over seven hundred pages. That shouldn’t scream “rational and in control of her passionate fury” to you.\r\nSo buckle up, buttercup. We’ve got some categories to run through.\r\nDISCLAIMER : This review is teeming with spoilers. It is also X-rated, due to the strength of my anger and the copious sex scenes in the source material.\r\nI HATE THESE CHARACTERS\r\nRhys makes me cringe harder than Renee Zellweger in Bridget Jones’ Diary. (At lea

In [12]:
review = coba.replace(",", "").replace("\n", "").replace("\r", "").replace("---", "").replace('|','').replace("!","").replace("//", "").replace("?","").replace("    ", " ").replace("  "," ").casefold()
review

"202.that’s the number of times that the word “mate” in any of its hellish forms is used in this book.two.hundred.and.two.https:emmareadstoomuch.wordpress.co...that statistic alone should be enough to justify this one star rating. if you feel satisfied enough to just remove yourself from this review run with that feeling. because i have like. seven pages of sh*t to complain about.that’s if i’m efficient.and i’ve never been an efficient reviewer. once. in my life. i counted two hundred (and two) uses of a word over seven hundred pages. that shouldn’t scream “rational and in control of her passionate fury” to you.so buckle up buttercup. we’ve got some categories to run through.disclaimer : this review is teeming with spoilers. it is also x-rated due to the strength of my anger and the copious sex scenes in the source material.i hate these charactersrhys makes me cringe harder than renee zellweger in bridget jones’ diary. (at least that movie has both colin firth and hugh grant to distrac

In [13]:
review_word = review.split(" ")
review_word

['202.that’s',
 'the',
 'number',
 'of',
 'times',
 'that',
 'the',
 'word',
 '“mate”',
 'in',
 'any',
 'of',
 'its',
 'hellish',
 'forms',
 'is',
 'used',
 'in',
 'this',
 'book.two.hundred.and.two.https:emmareadstoomuch.wordpress.co...that',
 'statistic',
 'alone',
 'should',
 'be',
 'enough',
 'to',
 'justify',
 'this',
 'one',
 'star',
 'rating.',
 'if',
 'you',
 'feel',
 'satisfied',
 'enough',
 'to',
 'just',
 'remove',
 'yourself',
 'from',
 'this',
 'review',
 'run',
 'with',
 'that',
 'feeling.',
 'because',
 'i',
 'have',
 'like.',
 'seven',
 'pages',
 'of',
 'sh*t',
 'to',
 'complain',
 'about.that’s',
 'if',
 'i’m',
 'efficient.and',
 'i’ve',
 'never',
 'been',
 'an',
 'efficient',
 'reviewer.',
 'once.',
 'in',
 'my',
 'life.',
 'i',
 'counted',
 'two',
 'hundred',
 '(and',
 'two)',
 'uses',
 'of',
 'a',
 'word',
 'over',
 'seven',
 'hundred',
 'pages.',
 'that',
 'shouldn’t',
 'scream',
 '“rational',
 'and',
 'in',
 'control',
 'of',
 'her',
 'passionate',
 'fury”',
 'to'

In [14]:
review_vocab = set(review_word)
review_vocab

{'rep',
 'caught',
 'guy.',
 'weapons',
 'nightmare',
 'banter',
 'girl...she',
 'book.-150',
 'later.at',
 'depiction',
 'talk.plotline',
 'representation',
 '“man',
 'things',
 'awful.',
 'book.two.hundred.and.two.https:emmareadstoomuch.wordpress.co...that',
 'out',
 'dialogue',
 'america’s',
 'way',
 '“women”',
 'threesomes;',
 'prettiness',
 'romantic',
 'douchey',
 'yes',
 'pages',
 'than',
 'not',
 'some',
 'eyelids.',
 'cliché.like',
 'always',
 'win.but',
 'female',
 'hand',
 'century',
 'closely.”',
 'once.',
 'wonder',
 'sort',
 'stupid.but',
 'whose',
 'somehow',
 'used',
 'males.',
 'winnowing',
 'beginning',
 'fulfillment',
 'handful',
 'passionate',
 'whatnot.',
 'get',
 'name',
 'adult',
 'seven',
 'men',
 'purple',
 'mere',
 'years',
 'unloved',
 'with',
 'nearby.',
 'heading',
 'certainly',
 'type',
 'absolutely',
 'misery',
 'consensus:',
 'her.morrigan',
 "couldn't",
 'omg...who',
 'personification',
 'respect',
 'extremely',
 'also',
 'you’re',
 'damaged',
 'length'

In [15]:
review = re.sub(r'\ by.*? on', "", review)
review = re.sub(r"\d\.\d", "", review)
review = re.sub(r"\d+\ [a-z]+\.*\ \d+", "", review)
review = review.replace(".", " ")
review = re.sub(r'\  *', " ", review)
review

"202 that’s the number of times that the word “mate” in any of its hellish forms is used in this book two hundred and two https:emmareadstoomuch wordpress co that statistic alone should be enough to justify this one star rating if you feel satisfied enough to just remove yourself from this review run with that feeling because i have like seven pages of sh*t to complain about that’s if i’m efficient and i’ve never been an efficient reviewer once in my life i counted two hundred (and two) uses of a word over seven hundred pages that shouldn’t scream “rational and in control of her passionate fury” to you so buckle up buttercup we’ve got some categories to run through disclaimer : this review is teeming with spoilers it is also x-rated due to the strength of my anger and the copious sex scenes in the source material i hate these charactersrhys makes me cringe harder than renee zellweger in bridget jones’ diary (at least that movie has both colin firth and hugh grant to distract me from re

In [16]:
print(len(data['review']))

21559


In [17]:
datas = []
for i in range(0, 21559):
    datas.append(data["review"][i]. replace(",", "").replace("\n", "").replace("\r", "").replace("---", "").replace('|','').replace("!","").replace("//", "").replace("?","").replace("    ", " ").replace("  "," ").casefold())

In [18]:
review = []
for i in range(0, 21559):
    review.append(data["review"][i].replace(",", "").replace("\n", "").replace("\r", "").replace("---", "").replace('|','').replace("!","").replace("//", "").replace("?","").replace("    ", " ").replace("  "," ").casefold())

In [19]:
reviews = []
for i in review:
    review = re.sub(r'\ by.*? on', "", i)
    review = re.sub(r"\d\.\d", "", review)
    review = re.sub(r"\d+\ [a-z]+\.*\ \d+", "", review)
    review = review.replace(".", " ")
    review = re.sub(r'\  *', " ", review)
    reviews.append(review)

In [20]:
data["reviews"] = reviews
data["reviews"]

0        202 that’s the number of times that the word “...
1        i need a black silk robe and a balcony to wear...
2        this book totally wrecked me in the best possi...
3        5 stars"rhys stepped off the stairs and took m...
4        i wish so much that i could tell you i loved t...
                               ...                        
21554    no one ask me about the book bc i literally re...
21555    i started the harry potter audio books at the ...
21556    “do not pity the dead harry pity the living an...
21557    harry potter changed my life if i hadn’t read ...
21558    lo que lloré con este libro no tiene nombre to...
Name: reviews, Length: 21559, dtype: object

In [21]:
data["review"][8461]

"5 STARS\r\nFirst off, I would like to sincerely thank Netgalley for the rare and precious ARC! I don't get them often, so imagine my thrill when I saw this baby!\r\nI will start out by saying that I am a huge fan of Sandra Brown and her gritty romance murder mysteries. But I want you to know that I'm not going to slap a 5 star rating up there just because I love the author's previous books, and because I am grateful for receiving this early release. (even though I am! ...grateful that is.) I going to do it because she has yet again delivered a gripping and emotionally charged story that led me down the primrose path, thinking I had it all worked out and then once again pulling the rug out from under my feet.\r\nThis was a very unique and intriguing storyline. One that would make a gripping movie. The kind you can on get from SB.\r\nLoved the characters. Even though I'm not a fan of reporters or newscasters of any kind, Kerra Bailey still managed to get under my skin and provoke a bit 

In [22]:
print("Banyak baris book title: {0}".format(data["book_title"].shape))
print("Banyak baris rating: {0}".format(data["book_rating"].shape))
print("Banyak baris genre: {0}".format(data["genre"].shape))
print("Banyak baris review: {0}".format(data["review"].shape))
print("Banyak baris, kolom data: {0}".format(data.shape))

Banyak baris book title: (21559,)
Banyak baris rating: (21559,)
Banyak baris genre: (21559,)
Banyak baris review: (21559,)
Banyak baris, kolom data: (21559, 16)


In [23]:
data[["book_title", "book_rating", "genre", "review"]].head()

,book_title,book_rating,genre,review
0,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",202.\r\nThat’s the number of times that the wo...
1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I need a black silk robe and a balcony to wear...
2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",THIS BOOK TOTALLY WRECKED ME. In the best poss...
3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","3.75 stars\r\n""Rhys stepped off the stairs and..."
4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I wish so much that I could tell you I loved t...


In [24]:
new = data[["ID", "book_title", "book_rating", "genre", "review"]]
new.to_csv("./parse_nlp.csv")

In [25]:
dataset = pd.read_csv("./parse_nlp.csv", delimiter=",")
dataset

,Unnamed: 0,ID,book_title,book_rating,genre,review
0,0,1,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",202.\r\nThat’s the number of times that the wo...
1,1,2,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I need a black silk robe and a balcony to wear...
2,2,3,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",THIS BOOK TOTALLY WRECKED ME. In the best poss...
3,3,4,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult","3.75 stars\r\n""Rhys stepped off the stairs and..."
4,4,5,A Court of Wings and Ruin,4.51,"Fantasy, Romance, Young Adult, New Adult",I wish so much that I could tell you I loved t...
...,...,...,...,...,...,...
21554,21554,21555,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction",no one ask me about the book bc i literally re...
21555,21555,21556,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction",I started the Harry Potter audio books at the ...
21556,21556,21557,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction","“Do not pity the dead, Harry. Pity the living,..."
21557,21557,21558,Harry Potter and the deathly hallows,4.62,"Fantasy, Young Adult, Fiction",Harry Potter changed my life. If I hadn’t read...


# Tokenisasi dan Mencari Vocabulary

In [26]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size = 0.2, random_state = 0)

In [27]:
len(train['book_title'].unique())

722

In [28]:
from nltk.tokenize import SpaceTokenizer
sTokenizer = SpaceTokenizer()
review_tokens = []
for i in train['review']:
    review_tokens.append(sTokenizer.tokenize(i))

In [29]:
train['review_tokens'] = review_tokens
train['review_tokens']

20663    [This, is, probably, the, second-to-last, wors...
14559    [4,5, Sterne!\r\n"Man, kann, sich, nicht, bis,...
548      [I, was, very, disappointed, with, this, book....
9569     [Enjoyed, this, one, a, lot., This, series, ke...
6484     [Sometimes, love, needs, a, second, chance, be...
                               ...                        
13123    [What's, the, worst, memory, you, have?, An, a...
19648    [IT’S, 2018, AND, AUTHORS, ARE, STILL, BASING,...
9845     [Book, 3, in, The, Stranje, House, series, did...
10799    [Obviously, very, influenced, by, Please, Kill...
2732     [Holy, crap,, this, book, was, INTENSE!, By, t...
Name: review_tokens, Length: 17247, dtype: object

In [30]:
vocab = []
for i in train['review_tokens']:
    vocab.append(set(i))

In [31]:
train['review_vocabs'] = vocab
train['review_vocabs']

20663    {it.\r\nA, ever, favour, of, worst, This, all,...
14559    {kann, Drachen, und, Band, warten,, erwarten,,...
548      {Veronica, him.\r\nHere, years., fight, There,...
9569     {summer, heartbreaking,, lot., but, Ugh,, on.,...
6484     {giddy-to-your-gut,, chemistry, me., swoon, SO...
                               ...                        
13123    {another, living, away?, he, That, explosive, ...
19648    {I’m, COACH, girl,, objects,, out, way, not, i...
9845     {be, House, disappoint., Sinclair, Book, did, ...
10799    {It, themes, insights, case, down, them, Me,, ...
2732     {unique., pacing, Gamma,, everything, way., fi...
Name: review_vocabs, Length: 17247, dtype: object

# Word Embedding with Word2Vec

In [32]:
word2vec_sg = Word2Vec(sentences=train['review_vocabs'], size=100, window=5, min_count=1, workers=4, sg=1, iter=10)

In [33]:
word2vec_sg.wv.similarity(w1="great",w2="good")

0.14515457

In [34]:
w1 = ["great"]
word2vec_sg.wv.most_similar (positive=w1,topn=6)

[('After', 0.8887320160865784),
 ('set', 0.8876858949661255),
 ('fiction', 0.8864606618881226),
 ('day', 0.8664805889129639),
 ('freaking', 0.8577721118927002),
 ('result', 0.8530902862548828)]

In [35]:
w1 = ["good",'fun','great']
w2 = ['bad']
word2vec_sg.wv.most_similar (positive=w1,negative=w2,topn=10)

[('suffer.\r\nAnd', 0.7151961326599121),
 ('peoples;', 0.7102429866790771),
 ('sweet.\r\nWhat', 0.7075954079627991),
 ('B.B.-8', 0.7051253914833069),
 ('Stars\r\nUnloved', 0.7047647833824158),
 ('failing;', 0.7037783265113831),
 ('(awwww', 0.7029929161071777),
 ('that.\r\nPlease', 0.699847400188446),
 ('thankfully)', 0.6996477842330933),
 ('gross.\r\n3.', 0.6979489326477051)]

In [36]:
word2vec_sg.wv.similarity(w1="good",w2="good")

1.0

In [37]:
train['review_vocabs'][0]

{'I’m',
 'girl...she',
 'depiction',
 'representation',
 'things',
 'awful.',
 'out',
 'dialogue',
 'ENTIRELY\r\nSJM',
 'pages',
 'closely.”',
 'whose',
 'males.',
 'winnowing',
 'beginning',
 'get',
 '(Feyre,',
 'mere',
 'Funniest',
 'quickly!!!',
 "couldn't",
 'also',
 'you’re',
 'DOING.',
 'length',
 'straight,',
 'Meeting',
 'it.',
 'ME',
 'genuinely',
 'comes',
 'CHARACTERS\r\nRhys',
 'Nesta',
 'do',
 'pretty',
 'suddenly,',
 'BOOK.\r\n----\r\n150',
 'into,',
 'NOVEL.\r\nIt',
 'scenes',
 'amusement',
 'deeply',
 'totally,',
 'chicken',
 'reason.”\r\n“It',
 'time.',
 'adolescence?',
 'end',
 'certain',
 'it.”',
 'Hugh',
 'cool',
 'sometimes',
 'war,',
 'Except',
 'Videos',
 'because',
 'my',
 'poisoned',
 'performer',
 'hilarious',
 'came',
 'ever',
 'loved',
 'guys,',
 'anyway).\r\nBecause,',
 'lot',
 'Days',
 'person,',
 'Manson',
 'joy',
 'depicting',
 'doll:',
 'HE',
 'millennium,',
 'It’s',
 'counted',
 'made',
 'through.\r\nDISCLAIMER',
 'doesn’t.',
 'mine,',
 'Sometimes',
 '

In [38]:
word2vec_sg.save("word2vec_model")
w2v = Word2Vec.load("word2vec_model")

In [39]:
book_data = []
for key,book in enumerate(train['review_vocabs']):
    if len(book) == 0:
        print(book, key)
    feature_vector = np.zeros((100,), dtype="float32")
    nwords = len(book)
    for token in book:
        feature_vector = np.add(feature_vector, w2v[token])
    
    feature_vector = np.divide(feature_vector, nwords)
    book_data.append({'book': data['book_title'][key], 'tags':book, 'vector':feature_vector})

In [40]:
book_vector_df = pd.DataFrame(book_data,columns=['book', 'tags', 'vector'])
book_vector_df.head()

,book,tags,vector
0,A Court of Wings and Ruin,"{it.\r\nA, ever, favour, of, worst, This, all,...","[0.123487525, 0.41904256, 0.39028224, -0.68712..."
1,A Court of Wings and Ruin,"{kann, Drachen, und, Band, warten,, erwarten,,...","[0.04987818, 0.046451785, 0.12555401, -0.29604..."
2,A Court of Wings and Ruin,"{Veronica, him.\r\nHere, years., fight, There,...","[0.14930308, 0.31695864, 0.47261885, -0.564460..."
3,A Court of Wings and Ruin,"{summer, heartbreaking,, lot., but, Ugh,, on.,...","[-0.0028334712, 0.32310733, 0.41649824, -0.522..."
4,A Court of Wings and Ruin,"{giddy-to-your-gut,, chemistry, me., swoon, SO...","[0.13035479, 0.2988041, 0.38313589, -0.5009357..."


In [41]:
book_vector_df.shape

(17247, 3)

# Book Recommendation

In [42]:
words_in_vocab= set(word2vec_sg.wv.index2word)

In [43]:
def get_cosine_similarity(row, search_vector):
    try:
        cosine_similarity = np.dot(row['vector'], search_vector)/linalg.norm(row['vector'])/linalg.norm(search_vector)
        return cosine_similarity
    except Exception as e:
        raise e

In [44]:
# Mencari buku dengan judul "A Court of Wings and Ruin"
book = input('Buku yang dicari: ')

Buku yang dicari: A Court of Wings and Ruin


In [45]:
book

'A Court of Wings and Ruin'

In [46]:
for i, j in enumerate(train['book_title']):
    if j == book:
        index_similar_book = i

search_query = train['review_vocabs'][index_similar_book]

In [47]:
search_vector = np.zeros((100,), dtype="float32")
for tag in search_query:
    if tag in words_in_vocab:
        search_vector = np.add(search_vector, w2v[tag])
        
book_vector_df['cosine_similarity'] = book_vector_df.apply(get_cosine_similarity, axis=1, args=(search_vector,))

In [48]:
df = book_vector_df.sort_values(by='cosine_similarity', ascending=False)[:10]

In [49]:
print("Maka buku yang direkomendasikan berdasarkan buku '{0}' adalah :".format(book))
df[['book', 'cosine_similarity']]

Maka buku yang direkomendasikan berdasarkan buku 'A Court of Wings and Ruin' adalah :


,book,cosine_similarity
17122,Mental Diplopia,1.000000
10281,Can't Hardly Breathe,0.997028
10251,That Inevitable Victorian Thing,0.996890
4109,Secrets in Death,0.996590
11510,Racer,0.996521
4053,We Are Okay,0.996436
7137,Goodnight from London,0.996430
11696,The Man in the Black Suit,0.996423
1183,Egomaniac,0.996421
9843,Refuge for Masterminds,0.996412
